#### OUMAR ALPHA YAYA CISSE

# Analyse de la forme :
    #- variable target : 'Outcome'
    #- Lignes et colonnes : 768 l, 9 c
    #- Types de variables : 8 variables continue, 1 variable discrète [1: 'positive', 0: 'negative']
    #- Analyse des valeurs manquantes : Aucune valeur manquante dans le data-set
    
# Analyse de fond :
##### ELIMINATION DES DONNES UNITILES
    #- Visualisation de la target variable : 500 personnes n'ont pas le diabète || 268 personnes ont le diabète
  
# Signification des Types variables : (Analyse de variable ) : 
 
    #- Variable continue    : certaines courbes suivent la lois de poisson et d'autre la loi normal les données n'ont pas                                     encore été standardisé
    
    #- Variable qualitative : une seule variable qualitative c'est la target ( Outcome ) les données on été encodé 
 
 
# RELATION TARGET / VARIABLES (Hypothèse ) : 

    #- TARGET / Variable continue : Les individus positives et negatives au diabète on des taux de glucose , BMI Différent
    
    #- TARGET / Age : Les individus de 21 a 28 ans sont les moins atteind du diabète | Les individus de 30 a 60 ans sont les                         plus atteind du diabète
    
# Relation variables / variables: ANALYSE PLUS DETAILLE (Corrélation ):

    #- Variable continue /  Variable continue : très peut de corrélation forte une seule corrélation dépasse les 50% celle entre age et Pregnancies 
    Nouvelle feature ajouté 'wasPregnant' indiquant si l'individus a été une fois enceinte
   
# NAN ANALYSE :
   ### AUCUNE VALEUR MANQUANTE


# HYPOTHESE :
    STUDENT TEST

## H0 : Les taux moyens de glucose , BMI sont egaux chez les individus positive et negative
    #- H1 : Les individus positives et negatives au diabète on des taux de glucose , BMI Différent
  ##### H1 ACEPPETE

# E D A

    # - MODULE 

In [ ]:
import pandas as pd 
import seaborn as sns
import numpy as np
from scipy.stats import ttest_ind
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split , learning_curve , GridSearchCV , RandomizedSearchCV
from sklearn.preprocessing import PolynomialFeatures , StandardScaler , OneHotEncoder , RobustScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix , classification_report, precision_recall_curve , accuracy_score , f1_score , recall_score
from sklearn.feature_selection import SelectKBest , f_classif , VarianceThreshold
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from imblearn.over_sampling import SMOTE

    # - CODE

In [ ]:
dbt =  pd.read_csv('../input/pima-indians-diabetes-database/diabetes.csv')
df = dbt.copy()
df['wasPregnant'] = df['Pregnancies'] >=1

In [ ]:
continue_variable = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI', 'DiabetesPedigreeFunction', 'Age']

In [ ]:
negatif_df = df[df['Outcome']==0]
positif_df = df[df['Outcome']==1]

In [ ]:
negatif_df_balanced = negatif_df.sample(positif_df.shape[0]) 

# PREPROCESSING

In [ ]:
prep_df = dbt.copy()

    #- ENCODAGE

In [ ]:
code = {
    True : 1,
    False : 0
}
def encodage(df):
    df['wasPregnant'] = df['wasPregnant'].map(code)
    return df

    #- FEATURE ENGENERING

In [ ]:
def feature_enge(df):  
    df = df.drop(['Pregnancies', 'SkinThickness' ] , axis=1)
    return df

    #- MODELE

In [ ]:
preprocess = make_pipeline(PolynomialFeatures(2, include_bias=False),SelectKBest(f_classif,k=4))
SVM_model = make_pipeline(preprocess,VarianceThreshold(threshold=1.5),RobustScaler(),SVC(random_state=0))

    #- PREPROCESSING

In [ ]:
Train_set , Test_set = train_test_split(prep_df,test_size=0.2)

In [ ]:
def preprocessing(df):
    df = feature_enge(df)
    #df = encodage(df)
    
    y = df['Outcome']
    X = df.drop('Outcome',axis=1)
    return X , y

In [ ]:
X_train , y_train = preprocessing(Train_set)
X_test , y_test = preprocessing(Test_set)

    #- SMOTE 

In [ ]:
over_sampling = SMOTE()

In [ ]:
X_train , y_train = over_sampling.fit_sample(X_train , y_train)
X_test , y_test = over_sampling.fit_sample(X_test , y_test)

    #- EVALUATION

In [ ]:
def evaluation(model):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)
    
    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))

    N , train_score , val_score = learning_curve(model,X_train,y_train,scoring='recall',cv=4, train_sizes=np.linspace(0.1,1,10))
    plt.figure(figsize=(12,9))
    plt.plot(N,train_score.mean(axis=1),label='train score')
    plt.plot(N,val_score.mean(axis=1),label='vald score')
    plt.legend()

In [ ]:
evaluation(SVM_model)

In [ ]:
#pd.DataFrame(SVM_TEST.feature_importances_, index=X_train.columns).plot.bar(figsize=(12,9))

In [ ]:
SVM_model

In [ ]:
hyper_params={
    'svc__C' : [1, 10,100 ,1000],
    'svc__gamma' : [1e-2,1e-1,1e-3],
    'pipeline__selectkbest__k' : [4, 2 , 3 , 5, 8 ,10],
    'pipeline__polynomialfeatures__degree' : [2 , 3 , 4 ],
    'variancethreshold__threshold' : [1.5 , 1.6 , 2.8 , 0.8]
}
grid = RandomizedSearchCV(SVM_model,hyper_params,scoring='f1',cv=4,n_iter=40)

In [ ]:
grid.fit(X_train,y_train)
evaluation(grid.best_estimator_)

# PRECISION RECALL CURVE

In [ ]:
 precision , recall , threshold= precision_recall_curve(y_test, grid.best_estimator_.decision_function(X_test))

In [ ]:
plt.figure(figsize=(12,9))
plt.plot(threshold,precision[:-1],label='precision')
plt.plot(threshold,recall[:-1],label='recall')
plt.legend()

In [ ]:
def model_final(X,model,threshold=0):
    return model.decision_function(X) > threshold

In [ ]:
y_pred = model_final(X_test,grid.best_estimator_,-0.2)

In [ ]:
accuracy_score(y_test,y_pred)

In [ ]:
f1_score(y_test,y_pred)

In [ ]:
recall_score(y_test,y_pred)